In [3]:
import os
import numpy as np
import mne
print (mne.__version__)
from scipy.io import loadmat, savemat
from mne.io import read_raw_egi, read_raw_fif
#import matplotlib.pyplot as plt
#%matplotlib notebook
import sys
from mne.preprocessing import ICA, create_eog_epochs, create_ecg_epochs
sys.path.insert(0, 'C:/Users/Antoine/github/MEG_pareidolia/python_scripts/Functions')
import MEG_pareidolia_utils
from MEG_pareidolia_utils import *

from mne.preprocessing import ICA, create_eog_epochs, create_ecg_epochs
from mne.io import read_raw_ctf, read_raw_fif
#from matplotlib.pyplot import plot as plt
#from matplotlib.pyplot import close as closefig
import mne
import numpy as np
from scipy.io import loadmat
import pandas as pd
import os
from IPython.display import clear_output
import time
import PARAMS
from PARAMS import *

1.2.2


In [4]:
def compute_FOOOF(epochs, function=psd_welch, tmin = None, tmax = None, max_freq=150, participant=None, bloc=None):
    #epochs are cropped as desire (tmin could be before '0', ex: -1.5, depending on the values used during epoching)
    epochs = epochs.apply_baseline((-1.5, -0.1))
    epochs.pick_types(meg=True, ref_meg=False)
    epochs = epochs.crop(tmin, tmax)
    print(epochs.get_data().shape)
    epo_data = epochs.get_data()
    exps = []
    df = pd.DataFrame(columns=['offset', 'exp', 'peaks_center', 'amps', 'width', 'participant', 'bloc', 'trial', 'electrode'])
    #This loop iterates for each epoch
    i=0
    for t in range(len(epo_data)):
        exps_elec = []
        for elec in range(len(epo_data[t])):
            offset, exp, cf, amp, width = FOOOF_aperiodic(epo_data[t][elec], 1200, precision=0.5, max_freq=max_freq, noverlap=None,
                                                          nperseg=None, nfft=None,
                                                          extended_returns=False, graph=False)
            df.loc[i] = offset, exp, cf, amp, width, participant, bloc, t, elec
            i+=1

    return df

In [46]:
def FOOOF_aperiodic(data, sf, precision=0.1, max_freq=80, min_freq=2,
                    noverlap=None,
                  nperseg=None, nfft=None,
                  extended_returns=False, graph=False):

    if nperseg is None:
        mult = 1/precision
        nfft = sf*mult
        nperseg = nfft
        noverlap = nperseg//10
    freqs1, psd = welch(data, sf, nfft=nfft,
                                     nperseg=nperseg, noverlap=noverlap)
    fm = FOOOF(peak_width_limits=[precision*2, 3], max_n_peaks=50,
               min_peak_height=0.3)

    freq_range = [(sf/len(data))*2, max_freq]
    fm.fit(freqs1, psd, freq_range)
    if graph is True:
        fm.report(freqs1, psd, freq_range)

    try:
        offset = fm.get_params('aperiodic_params')[0]
        exp = fm.get_params('aperiodic_params')[1]
    except:
        offset = 'NaN'
        exp = 'NaN'
    cf = [x[0] for x in fm.get_params('peak_params')]
    amp = [x[1] for x in fm.get_params('peak_params')]
    width = [x[2] for x in fm.get_params('peak_params')]
    return offset, exp, cf, amp, width, fm.fooofed_spectrum_, fm.freq_range, fm.freq_res

In [16]:
subj = '00'
run = 1
tmin=-1.5
tmax=-0.5

epochs_name, epochs_path = get_pareidolia_bids(FOLDERPATH, subj, 'pareidolia', run, stage = 'epo_RT')
epochs = mne.read_epochs(epochs_path)
epochs = epochs.apply_baseline((-2, -1.5))
epochs.pick_types(meg=True, ref_meg=False)
epochs = epochs.crop(tmin, tmax)
epochs_data = epochs.get_data()

Reading D:\PsychoPy_MEG\BIDS_data\sub-00\ses-recording\meg\sub-00_ses-recording_task-pareidolia_run-1_epo_RT.fif ...
    Found the data of interest:
        t =   -2000.00 ...    -500.00 ms
        0 CTF compensation matrices available
Adding metadata with 7 columns


C:\Users\Antoine\AppData\Local\Temp\ipykernel_15880\2667648498.py:7: RuntimeWarning: This filename (D:\PsychoPy_MEG\BIDS_data\sub-00\ses-recording\meg\sub-00_ses-recording_task-pareidolia_run-1_epo_RT.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(epochs_path)


52 matching events found
No baseline correction applied
0 projection items activated
Applying baseline correction (mode: mean)


In [47]:
max_freq = 80
noverlap=1
nperseg = 600
nfft = 1200

data = epochs_data[10][29]
offset, exp, cf, amp, width, det_spec, f_range, f_res = FOOOF_aperiodic(data, 1200, precision=1, max_freq=max_freq, min_freq=2, noverlap=noverlap,
                                                                          nperseg=nperseg, nfft=nfft,
                                                                          extended_returns=False, graph=False)

In [48]:
f_range

[2.0, 80.0]

In [35]:
freqs1, psd = welch(data, 1200, nfft=nfft,
                                     nperseg=nperseg, noverlap=noverlap)

In [45]:
detrend_psd = fm.fooofed_spectrum_
fm.freq_range
fm.freq_res
exp

1.2008110259374822